# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: 
- MSSV: 
- Lớp: 

# Import

In [2]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from itertools import combinations


# Đọc dữ liệu

In [3]:
train_1 = pd.read_csv('train.csv')
train_1 = train_1.sample(frac=1) # xáo trộn dữ liệu
# Đọc tập kiểm tra
test_1 = pd.read_csv('test.csv')

# Cài đặt hàm

In [4]:
# Cài đặt các hàm cần thiết ở đây

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [5]:
# Phần code cho yêu cầu 1a
reg1 = LinearRegression().fit(train_1.iloc[:, :11], train_1.iloc[:, -1]) # Lấy 11 thuộc tính đầu tiên của train_1

# Hiển thị công thức hồi quy tuyến tính
coef = pd.DataFrame({'Feature': train_1.iloc[:, :11].columns, 'Coefficient': reg1.coef_})

# Lấy hệ số chặn (intercept)
intercept = reg1.intercept_

# Tạo chuỗi biểu diễn công thức hồi quy tuyến tính
formula_parts = [f'{intercept:.3f}']  # Thêm hệ số chặn vào chuỗi công thức

for index, row in coef.iterrows():
    if row['Coefficient'] >= 0:
        formula_parts.append(f"+ {row['Coefficient']:.3f} * {row['Feature']}")
    else:
        formula_parts.append(f"- {abs(row['Coefficient']):.3f} * {row['Feature']}")

print(coef)
linear_formula = " ".join(formula_parts)
print("y(Salary) = ", linear_formula)



            Feature   Coefficient
0            Gender -23183.329508
1      10percentage    702.766792
2      12percentage   1259.018788
3       CollegeTier -99570.608141
4            Degree  18369.962450
5        collegeGPA   1297.532000
6   CollegeCityTier  -8836.727123
7           English    141.759939
8           Logical    145.742347
9             Quant    114.643313
10           Domain  34955.750405
y(Salary) =  49248.090 - 23183.330 * Gender + 702.767 * 10percentage + 1259.019 * 12percentage - 99570.608 * CollegeTier + 18369.962 * Degree + 1297.532 * collegeGPA - 8836.727 * CollegeCityTier + 141.760 * English + 145.742 * Logical + 114.643 * Quant + 34955.750 * Domain


In [6]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae = mean_absolute_error(test_1['Salary'], reg1.predict(test_1.iloc[:,:11]))
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 105052.52978823149


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} =  49248.090 - 23183.330 * Gender + 702.767 * 10percentage + 1259.019 * 12percentage - 99570.608 * CollegeTier + 18369.962 * Degree + 1297.532 * collegeGPA - 8836.727 * CollegeCityTier + 141.760 * English + 145.742 * Logical + 114.643 * Quant + 34955.750 * Domain$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [7]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
personality_features = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']

# Tạo DataFrame để lưu kết quả của k-fold Cross Validation cho mỗi đặc trưng
cv_results_b = pd.DataFrame(columns=['Feature', 'Mean_MAE'])

for feature in personality_features:
    # Tạo DataFrame chứa đặc trưng cần kiểm tra và cột 'Salary'
    train_b = train_1[[feature, 'Salary']]
    
    # Xây dựng mô hình hồi quy tuyến tính
    reg_b = LinearRegression().fit(train_b[[feature]], train_b['Salary'])
    
    # Sử dụng k-fold Cross Validation để tính Mean Absolute Error (MAE)
    mae_scores = -cross_val_score(reg_b, train_b[[feature]], train_b['Salary'], cv=5, scoring='neg_mean_absolute_error')
    mean_mae = np.mean(mae_scores)
    
    # Lưu kết quả vào DataFrame
    cv_results_b = pd.concat([cv_results_b, pd.DataFrame({'Feature': [feature], 'Mean_MAE': [mean_mae]})], ignore_index=True)

print(cv_results_b)
# Chọn đặc trưng tính cách tốt nhất
best_feature_b = cv_results_b.loc[cv_results_b['Mean_MAE'].idxmin()]['Feature']

print("Best personality feature:", best_feature_b)

# Đọc tập kiểm tra
test_1 = pd.read_csv('test.csv')




                 Feature       Mean_MAE
0      conscientiousness  124245.693661
1          agreeableness  123716.763359
2           extraversion  123936.547860
3            nueroticism  123553.104205
4  openess_to_experience  123889.396829
Best personality feature: nueroticism


In [8]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
reg_best = LinearRegression().fit(train_1[[best_feature_b]], train_1['Salary'])

In [9]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
# Dự đoán giá trị trên tập kiểm tra
predictions = reg_best.predict(test_1[[best_feature_b]])
# Tính Mean Absolute Error trên tập kiểm tra
mae_test = mean_absolute_error(test_1['Salary'], predictions)
print("Mean Absolute Error on test set:", mae_test)

# Hiển thị công thức hồi quy tuyến tính cho đặc trưng tốt nhất
intercept_best = reg_best.intercept_
linear_formula_best = f"{intercept_best:.3f} + {reg_best.coef_[0]:.3f} * {best_feature_b}"
print("Best linear formula:", linear_formula_best)

Mean Absolute Error on test set: 119361.91739987816
Best linear formula: 304647.553 + -16021.494 * nueroticism


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 304647.553 + -16021.494 * nueroticism$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [10]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
numerical_features = ['English', 'Logical', 'Quant']

# Tạo DataFrame để lưu kết quả của k-fold Cross Validation cho mỗi đặc trưng
cv_results_c = pd.DataFrame(columns=['Feature', 'Mean_MAE'])

for feature in numerical_features:
    # Tạo DataFrame chứa đặc trưng cần kiểm tra và cột 'Salary'
    train_c = train_1[[feature, 'Salary']]
    
    # Xây dựng mô hình hồi quy tuyến tính
    reg_c = LinearRegression().fit(train_c[[feature]], train_c['Salary'])
    
    # Sử dụng k-fold Cross Validation để tính Mean Absolute Error (MAE)
    mae_scores = -cross_val_score(reg_c, train_c[[feature]], train_c['Salary'], cv=5, scoring='neg_mean_absolute_error')
    mean_mae = np.mean(mae_scores)
    
    # Lưu kết quả vào DataFrame
    cv_results_c = pd.concat([cv_results_c, pd.DataFrame({'Feature': [feature], 'Mean_MAE': [mean_mae]})], ignore_index=True)

print(cv_results_c)
# Chọn đặc trưng ngoại ngữ, lô-gic, định lượng tốt nhất
best_feature_c = cv_results_c.loc[cv_results_c['Mean_MAE'].idxmin()]['Feature']
print("Best numerical feature:", best_feature_c)

   Feature       Mean_MAE
0  English  120774.495979
1  Logical  119944.177743
2    Quant  117217.426871
Best numerical feature: Quant


In [11]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
reg_best = LinearRegression().fit(train_1[[best_feature_c]], train_1['Salary'])

In [12]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
# Dự đoán giá trị trên tập kiểm tra
predictions = reg_best.predict(test_1[[best_feature_c]])

# Tính Mean Absolute Error trên tập kiểm tra
mae_test = mean_absolute_error(test_1['Salary'], predictions)
print("Mean Absolute Error on test set:", mae_test)

# Hiển thị công thức hồi quy tuyến tính cho đặc trưng tốt nhất
intercept_best = reg_best.intercept_
linear_formula_best = f"{intercept_best:.3f} + {reg_best.coef_[0]:.3f} * {best_feature_c}"
print("Best linear formula:", linear_formula_best)

Mean Absolute Error on test set: 108814.05968837196
Best linear formula: 117759.729 + 368.852 * Quant


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 117759.729 + 368.852 * Quant$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [13]:
# Trình bày các phần tìm ra mô hình
# Có 3 mô hình
# Danh sách 23 đặc trưng
all_features = ['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree', 'collegeGPA', 'CollegeCityTier',
                'English', 'Logical', 'Quant', 'Domain', 'ComputerProgramming', 'ElectronicsAndSemicon', 'ComputerScience',
                'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg', 'CivilEngg', 'conscientiousness', 'agreeableness',
                'extraversion', 'nueroticism', 'openess_to_experience']

def calculate_mae_for_feature_combination(data, feature_combination):
    # Chuyển tuple thành danh sách
    feature_list = list(feature_combination)
    
    # Tạo DataFrame chứa các đặc trưng cần kiểm tra và cột 'Salary'
    train_data = data[feature_list + ['Salary']]
    
    # Xây dựng mô hình hồi quy tuyến tính
    reg = LinearRegression().fit(train_data[feature_list], train_data['Salary'])
    
    # Sử dụng k-fold Cross Validation để tính Mean Absolute Error (MAE)
    mae_scores = -cross_val_score(reg, train_data[feature_list], train_data['Salary'], cv=5, scoring='neg_mean_absolute_error')
    mean_mae = np.mean(mae_scores)
    
    return mean_mae

def create_linear_formula(coefficients, intercept, feature_names):
    formula_parts = [f'{intercept:.3f}']  # Thêm hệ số chặn vào chuỗi công thức

    for coef, feature_name in zip(coefficients, feature_names):
        if coef >= 0:
            formula_parts.append(f"+ {coef:.3f} * {feature_name}")
        else:
            formula_parts.append(f"- {abs(coef):.3f} * {feature_name}")

    linear_formula = " ".join(formula_parts)
    return linear_formula

def find_best_feature_combination(data, features, num_features):
    best_mae = float('inf')  # Khởi tạo MAE tốt nhất ban đầu là vô cùng lớn
    best_combination = None   # Lưu lại tổ hợp tốt nhất
    
    # Tạo tất cả các tổ hợp có thể của num_features đặc trưng từ danh sách
    feature_combinations = combinations(features, num_features)
    
    for feature_combination in feature_combinations:
        # Tính MAE cho tổ hợp đặc trưng hiện tại
        current_mae = calculate_mae_for_feature_combination(data, feature_combination)
        
        # Kiểm tra xem tổ hợp hiện tại có tốt hơn tổ hợp tốt nhất hiện tại hay không
        if current_mae < best_mae:
            best_mae = current_mae
            best_combination = feature_combination
    
    # Tạo mô hình hồi quy tốt nhất với tổ hợp tốt nhất
    best_feature_names = list(best_combination)
    best_model = LinearRegression().fit(data[best_feature_names], data['Salary'])
    
    # Dự đoán giá trị trên tập kiểm tra
    predictions = best_model.predict(test_1[best_feature_names])

    # Tính Mean Absolute Error trên tập kiểm tra
    mae_test = mean_absolute_error(test_1['Salary'], predictions)
    
    # Tạo hàm y(Salary) từ mô hình tốt nhất
    linear_formula = create_linear_formula(best_model.coef_, best_model.intercept_, best_feature_names)
    
    return best_combination, mae_test, linear_formula

def model(num_features):
    best_combination, best_mae, linear_formula = find_best_feature_combination(train_1, all_features, num_features)
    print(f"Best feature combination: {best_combination}")
    print(f"Best Mean Absolute Error on set test: {best_mae:.3f}")
    print("Linear formula:", linear_formula)
    return best_combination, best_mae, linear_formula

## Thử nghiệm, so sánh các mô hình

In [14]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
mae_scores = []
for i in range(2,5):
    start_time = time.time()
    mae=0
    if i == 2:
        _,mae,_ = model(2)
    elif i == 3:
        _,mae,_ = model(3)
    elif i == 4:
        _,mae,_ = model(4)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"Thời gian chạy: {execution_time:.3f} giây\n\n")
    mae_scores.append(mae)

Best feature combination: ('10percentage', 'Quant')
Best Mean Absolute Error on set test: 106022.179
Linear formula: -21420.874 + 2178.347 * 10percentage + 310.968 * Quant
Thời gian chạy: 3.993 giây


Best feature combination: ('10percentage', 'Quant', 'ComputerProgramming')
Best Mean Absolute Error on set test: 104794.139
Linear formula: -47200.597 + 2195.725 * 10percentage + 284.155 * Quant + 107.263 * ComputerProgramming
Thời gian chạy: 28.544 giây


Best feature combination: ('10percentage', 'Quant', 'ComputerProgramming', 'ComputerScience')
Best Mean Absolute Error on set test: 103141.108
Linear formula: -35516.003 + 2141.270 * 10percentage + 272.858 * Quant + 144.423 * ComputerProgramming - 156.966 * ComputerScience
Thời gian chạy: 145.120 giây




In [15]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
model(np.argmin(mae_scores)+2)


Best feature combination: ('10percentage', 'Quant', 'ComputerProgramming', 'ComputerScience')
Best Mean Absolute Error on set test: 103141.108
Linear formula: -35516.003 + 2141.270 * 10percentage + 272.858 * Quant + 144.423 * ComputerProgramming - 156.966 * ComputerScience


(('10percentage', 'Quant', 'ComputerProgramming', 'ComputerScience'),
 103141.10846961295,
 '-35516.003 + 2141.270 * 10percentage + 272.858 * Quant + 144.423 * ComputerProgramming - 156.966 * ComputerScience')

In [16]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
print(mae_test)

108814.05968837196


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} =  -35516.003 + 2141.270 * 10percentage + 272.858 * Quant + 144.423 * ComputerProgramming - 156.966 * ComputerScience$$